In [ ]:
import pandas as pd
import os
import csv
import calendar
import datetime
os.getcwd()
os.chdir('D:/Desktop/Flex 3/Grad Case Study/Data/')
os.getcwd()

### Filter Data

In [8]:
#ACCT_BAL = 47828339 ~ 47.8 million total rows = 40022484 ~ 40 million filtered rows
#TRAN_sum = 42624951 ~ 42.6 million total rows = 31769701 ~ 31.7 million filtered rows
#HH_sum = 77173079 ~ 77.2 million total rows = 32584691  ~ 32.6 million filtered rows
#CUST_sum = 98118047 ~ 98.1 million total rows = 37871352 ~ 37.9 million filtered rows

#ACCT_BAL = 3873981 ~ 3.9 million unique keys (I_REC_KEY)   
#TRAN_sum = 4278082 ~ 4.3 million unique rows (I_REC_KEY, ACCT_KEY)
#HH_sum = 6115024 ~ 6.1 million unique keys (HH_KEY)
#cust_sum = 8794824 ~ 8.8 million unique keys (CUST_KEY)
#HH_sum = 5648857 ~ 5.65 million filtered unique keys (HH_KEY)
#REF table unique_keys = 3214696 ~ 3.2 million

#### Filter only Checking products from accounts summary file

In [3]:
#Reading Account Balance file
ACCT_BAL=pd.read_csv('/UC_CoBA_NJB/UC_CoBA_ACCT_BAL_MONTHLY_15.txt', header=0,sep='|')
ACCT_BAL

,I_REC_KEY,DATE_KEY,AVG_MONTHLY_BAL_AMT,LAST_STMT_BAL_AMT,MRKTG_PROD_NAME
0,8923024,31OCT2018,2653.0,2487.0,REGULAR BUS CHECKING
1,8923024,31DEC2018,2794.0,4160.0,REGULAR BUS CHECKING
2,8923024,30NOV2018,2578.0,3626.0,REGULAR BUS CHECKING
3,8923024,31JAN2019,2332.0,2374.0,REGULAR BUS CHECKING
4,8923024,28FEB2019,2357.0,2648.0,REGULAR BUS CHECKING
...,...,...,...,...,...
47828334,159358854,31DEC2019,0.0,0.0,5/3 FREE CHECKING
47828335,159358693,31DEC2019,16.0,0.0,5/3 FREE CHECKING
47828336,159358870,31DEC2019,0.0,0.0,5/3 FREE CHECKING
47828337,159358595,31DEC2019,661.0,0.0,ENHANCED CHECKING


In [4]:
#Count number of unique products
ACCT_BAL.MRKTG_PROD_NAME.nunique()

58

In [4]:
#Save unique products
checking_account=pd.DataFrame(ACCT_BAL.MRKTG_PROD_NAME.unique())
#checking_account.to_csv("identify_checking_accounts.csv", index=False, header=True)

In [5]:
#Filter only Checking accounts
checking_prod=["CHECKING","CKG","CHKNG","CHKG"]
ACCT_prod_filtered=ACCT_BAL[ACCT_BAL['MRKTG_PROD_NAME'].str.contains('|'.join(checking_prod))]

In [10]:
#cross check number of filtered checking accounts
ACCT_prod_filtered.MRKTG_PROD_NAME.nunique()

25

In [6]:
#Store unique products as list
acct_prod_unique_keys=ACCT_prod_filtered['I_REC_KEY'].unique().tolist()
#1)End of Product filtering

#### Filter 'Retail', 'Consumer' from Household summary file

In [ ]:
#Reading HH file
HH_sum_table=pd.read_csv('UC_CoBA_HH_SUMM_MONTHLY_15.txt', header=0,sep='|',usecols=['HH_KEY','HH_SEG_CODE','HH_TYPE_CODE'])

In [ ]:
#Filter Retail Consumer and convert it to list
HH_seg_unique_keys=['R']
HH_type_unique_keys=['C']
HH_seg_filtered=HH_sum_table[HH_sum_table['HH_TYPE_CODE'].isin(HH_type_unique_keys)]
HH_seg_type_filtered=HH_seg_filtered[HH_seg_filtered['HH_SEG_CODE'].isin(HH_seg_unique_keys)]
HH_filtered_keys=HH_seg_type_filtered['HH_KEY'].unique().tolist()
#1)End of HH Seg filtering

#### Filter the Reference file by 'Retail', 'Consumer' with Checking Products

In [2]:
#Reading the Reference file
CUST_HH_REF=pd.read_csv('UC_CoBA_ACCT_CUST_HH_XREF.txt', header=0,sep='|')
CUST_HH_REF

,I_REC_KEY,ACCT_KEY,CURR_CUST_KEY,CURR_HH_KEY
0,39155074,16110071,32777172.0,6520155.0
1,14005888,16256538,34896115.0,3407771.0
2,131963766,17317526,20040290.0,6593223.0
3,10190016,16298146,33908034.0,964223.0
4,131777109,17303491,61578406.0,6523463.0
...,...,...,...,...
20935733,133409944,70594128,19320352.0,12918952.0
20935734,13227827,9098557,24558970.0,4093352.0
20935735,155309730,80500877,69230532.0,75736723.0
20935736,135209687,71924766,19320749.0,13574886.0


In [ ]:
#Filter the Ref file by 1) Products
ref_prod_filtered=CUST_HH_REF[CUST_HH_REF['I_REC_KEY'].isin(acct_prod_unique_keys)]
ref_prod_filtered.shape

In [ ]:
#3) Filter the Ref file by 2) HH Segments
REF_prod_seg_filtered=ref_prod_filtered[ref_prod_filtered['CURR_HH_KEY'].isin(HH_filtered_keys)]

In [ ]:
#Finally, count the number of unique accounts filtered
REF_prod_seg_filtered['ACCT_KEY'].nunique()

In [37]:
#Save the file
REF_prod_seg_filtered.to_csv("final_filtered_keys.csv", index=False, header=True)
REF_prod_seg_filtered.to_csv("final_filtered_keys.txt", index=False, header=True)
REF_prod_seg_filtered

,I_REC_KEY,ACCT_KEY,CURR_CUST_KEY,CURR_HH_KEY
5,13970302,16198496,33452982.0,3552900.0
6,13095040,16217550,59936676.0,3932333.0
7,12861899,16248733,60770157.0,4026773.0
8,14326125,16249054,32359271.0,3792490.0
10,14354993,16306848,23563952.0,8071512.0
...,...,...,...,...
20935072,66785729,42186935,19285086.0,15400972.0
20935073,9720987,1895448,19287980.0,73982670.0
20935074,97548809,56591739,19291002.0,76036060.0
20935079,14694198,17039189,19280620.0,3608967.0


This Code is to get last dates of observation for a account and then to create targets (Customer likely to churn). Any account which is not present in the last date of the data files are flagged to abe attrite

### Mark Attrite customers

In [2]:
keys = pd.read_csv('final_filtered_keys.csv')
keys.shape

In [ ]:
# Dividing number of keys into 3 parts (30%, 35%, 35%) 964431+1125170+1125170
keys_new = keys.sample(frac=0.3)
keys_new_rest = keys[~keys.index.isin(keys_new.index)]
keys_new_rest1 = keys_new_rest.sample(frac=0.5)
keys_new_rest2 = keys_new_rest[~keys_new_rest.index.isin(keys_new_rest1.index)]

In [ ]:
accts = pd.read_table('UC_CoBA_ACCT_BAL_MONTHLY_15.txt', usecols= ['I_REC_KEY','DATE_KEY'] ,sep="|")
accts.shape

In [ ]:
joined_file1 = pd.merge(accts, 
                  keys_new,
                  left_on='I_REC_KEY',
                  right_on='I_REC_KEY',
                  how='inner')

In [ ]:
joined_file2 = pd.merge(accts, 
                  keys_new_rest1,
                  left_on='I_REC_KEY',
                  right_on='I_REC_KEY',
                  how='inner')

In [ ]:
joined_file3 = pd.merge(accts, 
                  keys_new_rest2,
                  left_on='I_REC_KEY',
                  right_on='I_REC_KEY',
                  how='inner')

In [3]:
date_format = "%d%b%Y"

In [ ]:
joined_file1 = joined_file1.drop(columns=['ACCT_KEY','CURR_CUST_KEY','CURR_HH_KEY'], axis=1)
joined_file2 = joined_file2.drop(columns=['ACCT_KEY','CURR_CUST_KEY','CURR_HH_KEY'], axis=1)
joined_file3 = joined_file3.drop(columns=['ACCT_KEY','CURR_CUST_KEY','CURR_HH_KEY'], axis=1)

Run Below codes in the sequence mentioned in comments 

In [ ]:
joined_file1['newdate'] = [datetime.datetime.strptime(d, date_format) for d in joined_file1["DATE_KEY"]] # Run1 

In [ ]:
joined_file2['newdate'] = [datetime.datetime.strptime(d, date_format) for d in joined_file2["DATE_KEY"]] # Run4

In [ ]:
joined_file3['newdate'] = [datetime.datetime.strptime(d, date_format) for d in joined_file3["DATE_KEY"]] # Run7

In [ ]:
grouped_acct1 = joined_file1.groupby(['I_REC_KEY']).apply(pd.DataFrame.sort_values, 'newdate') # Run2

In [ ]:
grouped_acct2 = joined_file2.groupby(['I_REC_KEY']).apply(pd.DataFrame.sort_values, 'newdate') #Run5

In [ ]:
grouped_acct3 = joined_file3.groupby(['I_REC_KEY']).apply(pd.DataFrame.sort_values, 'newdate')

In [ ]:
last_date1 = grouped_acct1.groupby(['I_REC_KEY']).last() # Run3

In [ ]:
last_date2 = grouped_acct2.groupby(['I_REC_KEY']).last() # Run6

In [ ]:
last_date3 = grouped_acct3.groupby(['I_REC_KEY']).last() #Run8 

In [ ]:
flag1 = [] 
for value in last_date1["DATE_KEY"]: 
    if value == "31DEC2019": 
        flag1.append(0) 
    else: 
        flag1.append(1)

last_date1["flag"] = flag1   

In [ ]:
flag2 = [] 
for value in last_date2["DATE_KEY"]: 
    if value == "31DEC2019": 
        flag2.append(0) 
    else: 
        flag2.append(1)

last_date2["flag"] = flag2

In [ ]:
flag3 = [] 
for value in last_date1["DATE_KEY"]: 
    if value == "31DEC2019": 
        flag3.append(0) 
    else: 
        flag3.append(1)

last_date3["flag"] = flag3

In [ ]:
last_date = last_date1.append(last_date2)
last_date = last_date.append(last_date3)
last_date = last_date.rename(columns={"DATE_KEY":"lastdate","newdate":"lastdateindate","flag":"attritionflag"})

In [ ]:
last_date.to_csv('attritionflags.csv') # Should have (3214771, 4)

### Create Target Flag 

In [ ]:
accts.DATE_KEY.unique()
accts.shape, accts.head()

In [ ]:
attrition_flag = pd.read_csv('attritionflags.csv')

In [ ]:
# Load accts and attrition flag file before 

def target_func2(for_month,months,rate):         ### rate =1 for target rate , rate anything other than 1 will output the file
    acct = accts[accts['DATE_KEY']== for_month]
    
    for_date = datetime.datetime.strptime(for_month, date_format)
    month = for_date.month
    file_month = month
    year = for_date.year + month // 12
    month = month % 12 + 1
    day = max(1, calendar.monthrange(year,month)[1])
    
    last_month = datetime.datetime(year,month,day,0,0,0)
    last_month = last_month.strftime('%d%b%Y').upper()
    
    attrition = attrition_flag[attrition_flag['attritionflag']==1]
    attrition = attrition[attrition['lastdate']==last_month]
    file = pd.merge(acct,
                   attrition,
                   left_on = 'I_REC_KEY',
                   right_on = 'I_REC_KEY',
                   how = 'left')
    target = [] 
    for value in file["attritionflag"]: 
        if value == 1: 
            target.append(1) 
        else: 
            target.append(0) 
    file['target']=target
    file['month']=file_month
    file = file.drop(columns=['lastdate','lastdateindate','attritionflag'],axis=1)
    
    target_rate = file[file['target']==1].shape[0]/file.shape[0]
    if rate == 1: 
        return target_rate 
    else: 
        return file

In [ ]:
nov_file = target_func2('30NOV2018',1,0)
dec_file = target_func2('31DEC2018',1,0)
jan_file = target_func2('31JAN2019',1,0)
feb_file = target_func2('28FEB2019',1,0)
mar_file = target_func2('31MAR2019',1,0)
apr_file = target_func2('30APR2019',1,0)
may_file = target_func2('31MAY2019',1,0)
jun_file = target_func2('30JUN2019',1,0)
jul_file = target_func2('31JUL2019',1,0)
aug_file = target_func2('31AUG2019',1,0)
sep_file = target_func2('30SEP2019',1,0)
oct_file = target_func2('31OCT2019',1,0)

In [ ]:
df = nov_file.append([dec_file, jan_file,feb_file,mar_file,apr_file,may_file,jun_file,jul_file,aug_file,sep_file,oct_file])

In [ ]:
## Sampling and building final dataframe

final_df = pd.DataFrame(columns=['I_REC_KEY','DATE_KEY','target','month'])

for i in range(1,13):
    if i == 1:
        sam_key = (df[df['month']==i]).sample(250000)
        final_df = final_df.append(sam_key)
    else:
        df_temp = df[df['month']==i]
        sam_key = df_temp[~df_temp.I_REC_KEY.isin(final_df['I_REC_KEY'])].sample(250000)
        final_df = final_df.append(sam_key)

In [ ]:
final_df.to_csv('Finaldata_4_EDA.csv')

The Data is prepared for EDA now 

In [1]:
This Code is to get last dates of observation for a account and then to create targets (Customer likely to churn). Any account which is not present in the last date of the data files are flagged to abe attrite

### Mark Attrite customers

import pandas as pd
import os
import csv
import calendar
import datetime
os.getcwd()
os.chdir('D:\Cincinnati\Classes\9. Graduate Case studies')
os.getcwd()

keys = pd.read_csv('final_filtered_keys.csv')
keys.shape

# Dividing number of keys into 3 parts (30%, 35%, 35%) 964431+1125170+1125170
keys_new = keys.sample(frac=0.3)
keys_new_rest = keys[~keys.index.isin(keys_new.index)]
keys_new_rest1 = keys_new_rest.sample(frac=0.5)
keys_new_rest2 = keys_new_rest[~keys_new_rest.index.isin(keys_new_rest1.index)]

accts = pd.read_table('UC_CoBA_ACCT_BAL_MONTHLY_15.txt', usecols= ['I_REC_KEY','DATE_KEY'] ,sep="|")
accts.shape

joined_file1 = pd.merge(accts, 
                  keys_new,
                  left_on='I_REC_KEY',
                  right_on='I_REC_KEY',
                  how='inner')

joined_file2 = pd.merge(accts, 
                  keys_new_rest1,
                  left_on='I_REC_KEY',
                  right_on='I_REC_KEY',
                  how='inner')

joined_file3 = pd.merge(accts, 
                  keys_new_rest2,
                  left_on='I_REC_KEY',
                  right_on='I_REC_KEY',
                  how='inner')

date_format = "%d%b%Y"

joined_file1 = joined_file1.drop(columns=['ACCT_KEY','CURR_CUST_KEY','CURR_HH_KEY'], axis=1)
joined_file2 = joined_file2.drop(columns=['ACCT_KEY','CURR_CUST_KEY','CURR_HH_KEY'], axis=1)
joined_file3 = joined_file3.drop(columns=['ACCT_KEY','CURR_CUST_KEY','CURR_HH_KEY'], axis=1)

Run Below codes in the sequence mentioned in comments 

joined_file1['newdate'] = [datetime.datetime.strptime(d, date_format) for d in joined_file1["DATE_KEY"]] # Run1 

joined_file2['newdate'] = [datetime.datetime.strptime(d, date_format) for d in joined_file2["DATE_KEY"]] # Run4

joined_file3['newdate'] = [datetime.datetime.strptime(d, date_format) for d in joined_file3["DATE_KEY"]] # Run7

grouped_acct1 = joined_file1.groupby(['I_REC_KEY']).apply(pd.DataFrame.sort_values, 'newdate') # Run2

grouped_acct2 = joined_file2.groupby(['I_REC_KEY']).apply(pd.DataFrame.sort_values, 'newdate') #Run5

grouped_acct3 = joined_file3.groupby(['I_REC_KEY']).apply(pd.DataFrame.sort_values, 'newdate')

last_date1 = grouped_acct1.groupby(['I_REC_KEY']).last() # Run3

last_date2 = grouped_acct2.groupby(['I_REC_KEY']).last() # Run6

last_date3 = grouped_acct3.groupby(['I_REC_KEY']).last() #Run8 

flag1 = [] 
for value in last_date1["DATE_KEY"]: 
    if value == "31DEC2019": 
        flag1.append(0) 
    else: 
        flag1.append(1)

last_date1["flag"] = flag1   

flag2 = [] 
for value in last_date2["DATE_KEY"]: 
    if value == "31DEC2019": 
        flag2.append(0) 
    else: 
        flag2.append(1)

last_date2["flag"] = flag2

flag3 = [] 
for value in last_date1["DATE_KEY"]: 
    if value == "31DEC2019": 
        flag3.append(0) 
    else: 
        flag3.append(1)

last_date3["flag"] = flag3

last_date = last_date1.append(last_date2)
last_date = last_date.append(last_date3)
last_date = last_date.rename(columns={"DATE_KEY":"lastdate","newdate":"lastdateindate","flag":"attritionflag"})

last_date.to_csv('attritionflags.csv') # Should have (3214771, 4)

### Create Target Flag 

accts.DATE_KEY.unique()
accts.shape, accts.head()

attrition_flag = pd.read_csv('attritionflags.csv')

# Load accts and attrition flag file before 

def target_func2(for_month,months,rate):         ### rate =1 for target rate , rate anything other than 1 will output the file
    acct = accts[accts['DATE_KEY']== for_month]
    
    for_date = datetime.datetime.strptime(for_month, date_format)
    month = for_date.month
    file_month = month
    year = for_date.year + month // 12
    month = month % 12 + 1
    day = max(1, calendar.monthrange(year,month)[1])
    
    last_month = datetime.datetime(year,month,day,0,0,0)
    last_month = last_month.strftime('%d%b%Y').upper()
    
    attrition = attrition_flag[attrition_flag['attritionflag']==1]
    attrition = attrition[attrition['lastdate']==last_month]
    file = pd.merge(acct,
                   attrition,
                   left_on = 'I_REC_KEY',
                   right_on = 'I_REC_KEY',
                   how = 'left')
    target = [] 
    for value in file["attritionflag"]: 
        if value == 1: 
            target.append(1) 
        else: 
            target.append(0) 
    file['target']=target
    file['month']=file_month
    file = file.drop(columns=['lastdate','lastdateindate','attritionflag'],axis=1)
    
    target_rate = file[file['target']==1].shape[0]/file.shape[0]
    if rate == 1: 
        return target_rate 
    else: 
        return file

nov_file = target_func2('30NOV2018',1,0)
dec_file = target_func2('31DEC2018',1,0)
jan_file = target_func2('31JAN2019',1,0)
feb_file = target_func2('28FEB2019',1,0)
mar_file = target_func2('31MAR2019',1,0)
apr_file = target_func2('30APR2019',1,0)
may_file = target_func2('31MAY2019',1,0)
jun_file = target_func2('30JUN2019',1,0)
jul_file = target_func2('31JUL2019',1,0)
aug_file = target_func2('31AUG2019',1,0)
sep_file = target_func2('30SEP2019',1,0)
oct_file = target_func2('31OCT2019',1,0)

df = nov_file.append([dec_file, jan_file,feb_file,mar_file,apr_file,may_file,jun_file,jul_file,aug_file,sep_file,oct_file])

## Sampling and building final dataframe

final_df = pd.DataFrame(columns=['I_REC_KEY','DATE_KEY','target','month'])

for i in range(1,13):
    if i == 1:
        sam_key = (df[df['month']==i]).sample(250000)
        final_df = final_df.append(sam_key)
    else:
        df_temp = df[df['month']==i]
        sam_key = df_temp[~df_temp.I_REC_KEY.isin(final_df['I_REC_KEY'])].sample(250000)
        final_df = final_df.append(sam_key)

final_df.to_csv('Finaldata_4_EDA.csv')

The Data is prepared for EDA now import pandas as pd
import numpy as np
from datetime import datetime

### Filter base data for training model and create derived variables

In [2]:
#Read the sampled file for 3 columns:I_REC_KEY and obs_date and target
# create a list of sampled keys

EDA=pd.read_csv('Finaldata_4_EDA.csv',usecols=['I_REC_KEY','obs_date','target'])
keys=EDA.I_REC_KEY.unique().tolist()
EDA.head()

,I_REC_KEY,target,obs_date
0,141650292,0,31JAN2019
1,47743555,0,31JAN2019
2,150557313,0,31JAN2019
3,10480411,0,31JAN2019
4,127664122,0,31JAN2019


In [3]:
#Filter tran_summ for keys from the merged file

df_sample = pd.read_table('UC_CoBA_TRAN_SUMM_MONTHLY_15.txt', nrows=10)
df_sample_size = df_sample.memory_usage(index=True).sum()

my_chunk = (1000000000 / df_sample_size)/10
my_chunk = int(my_chunk//1) # we get the integer part
print (my_chunk)

df_chunk = pd.read_csv('UC_CoBA_TRAN_SUMM_MONTHLY_15.txt', chunksize=my_chunk,sep="|")
chunk_list = []  # append each chunk df here
 
# Each chunk is in df format
for chunk in df_chunk: 
    # perform data filtering
    chunk_filter = chunk[chunk['I_REC_KEY'].isin(keys)]
   
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk_filter)
   
# concat the list into dataframe
tran = pd.concat(chunk_list)

print(tran.shape)
tran.head()

480769
(31357764, 17)


,ACCT_KEY,I_REC_KEY,DATE_KEY,ACH_IN_MTD_AMT,ACH_IN_MTD_QTY,ACH_OUT_MTD_AMT,ACH_OUT_MTD_QTY,CHECK_CASHING_MTD_AMT,CHECK_CASHING_MTD_QTY,CHK_WRITTEN_MTD_AMT,CHK_WRITTEN_MTD_QTY,DEBIT_CARD_MTD_AMT,DEBIT_CARD_MTD_QTY,MOBILE_IMD_DEP_AMT,MOBILE_IMD_DEP_QTY,MOBILE_STD_DEP_AMT,MOBILE_STD_DEP_QTY
1,3683895,9023693,31OCT2018,5392.35,2,352.61,4,0.0,0,258.00,1,0.00,0,0.0,0,0.0,0
2,2223868,9035845,31OCT2018,6205.11,4,67.93,2,0.0,0,223.75,3,1593.74,2,0.0,0,0.0,0
3,3400641,9676268,31OCT2018,0.00,0,4205.93,4,0.0,0,2500.00,1,350.04,3,0.0,0,0.0,0
4,2599780,9350982,31OCT2018,5130.46,3,8341.19,17,0.0,0,1200.00,2,3144.60,73,0.0,0,2300.0,3
5,3624223,9036014,31OCT2018,1517.64,2,6.00,1,0.0,0,339.06,3,555.03,14,0.0,0,0.0,0


In [ ]:
print(tran.columns)

In [ ]:
#Merge on tran_summ with sampled file
tran_eda=pd.merge(tran,EDA[['I_REC_KEY','obs_date','target']],on=['I_REC_KEY'])
tran_eda.shape

In [ ]:
# convert obs date and tran date to date type and sort the table by I_REC_KEY and tran date
tran_eda['DATE']=pd.to_datetime(tran_eda.DATE_KEY)
tran_eda['obs_date2']=pd.to_datetime(tran_eda.obs_date)
tran_eda=tran_eda.sort_values(['I_REC_KEY','DATE'])
print(tran_eda.shape)
#tran_eda.to_csv('tran_target.txt',header=True,index=False)

#### Create New variables in Transaction summary file
% difference of amount ($) with last month for each account<br>
difference between number of swipes with last month for each account

In [ ]:
#Create new variables - % diff of Amount and diff of Quantity

#ACH_IN
%timeit tran_eda['%diff_ACH_IN']=tran_eda.groupby(['ACCT_KEY']).ACH_IN_MTD_AMT.pct_change().replace((np.inf,-np.inf,np.nan), 0)
tran_eda['diff_ACH_IN_QTY']=tran_eda.groupby(['ACCT_KEY']).ACH_IN_MTD_QTY.diff().replace((np.inf,-np.inf,np.nan), 0)

In [ ]:
#ACH_OUT
%timeit tran_eda['%diff_ACH_OUT']=tran_eda.groupby(['ACCT_KEY']).ACH_OUT_MTD_AMT.pct_change().replace((np.inf,-np.inf,np.nan), 0)
tran_eda['diff_ACH_OUT_QTY']=tran_eda.groupby(['ACCT_KEY']).ACH_OUT_MTD_QTY.diff().replace((np.inf,-np.inf,np.nan), 0)

In [ ]:
#CHECK_CASHING
%timeit tran_eda['%diff_CHECK_CASHING']=tran_eda.groupby(['ACCT_KEY']).CHECK_CASHING_MTD_AMT.pct_change().replace((np.inf,-np.inf,np.nan), 0)
tran_eda['diff_CHECK_CASHING_QTY']=tran_eda.groupby(['ACCT_KEY']).CHECK_CASHING_MTD_QTY.diff().replace((np.inf,-np.inf,np.nan), 0)

In [ ]:
#CHECK_WRITTEN
%timeit tran_eda['%diff_CHECK_WRITTEN']=tran_eda.groupby(['ACCT_KEY']).CHK_WRITTEN_MTD_AMT.pct_change().replace((np.inf,-np.inf,np.nan), 0)
tran_eda['diff_CHECK_WRITTEN']=tran_eda.groupby(['ACCT_KEY']).CHK_WRITTEN_MTD_QTY.diff().replace((np.inf,-np.inf,np.nan), 0)

In [ ]:
#DEBIT_CARD
%timeit tran_eda['%diff_DEBIT_CARD']=tran_eda.groupby(['ACCT_KEY']).DEBIT_CARD_MTD_AMT.pct_change().replace((np.inf,-np.inf,np.nan), 0)
tran_eda['diff_DEBIT_CARD_QTY']=tran_eda.groupby(['ACCT_KEY']).DEBIT_CARD_MTD_QTY.diff().replace((np.inf,-np.inf,np.nan), 0)

In [ ]:
#Check written per trans, Debit swipe per trans
%timeit tran_eda['CHK_WRITTEN_per_trans']=tran_eda['CHK_WRITTEN_MTD_AMT'].div(tran['CHK_WRITTEN_MTD_QTY']).replace((np.inf,-np.inf,np.nan), 0)
tran_eda['DEBIT_CARD_per_trans']=tran_eda['DEBIT_CARD_MTD_AMT'].div(tran['DEBIT_CARD_MTD_QTY']).replace((np.inf,-np.inf,np.nan), 0)

In [ ]:
tran_eda.columns

In [ ]:
print(tran_eda.shape)
tran_eda.to_csv('Trans_derived_var.txt',header=True,index=False)
tran_eda.head()

#### Create variable for number of inactive months 

In [ ]:
#create a flag column to get last tran date such that obs date>=tran date
tran_eda['date_flag'] = np.where(tran_eda['obs_date2']>=tran_eda['DATE'], 1, 0)

#Filter the flagged observations
tran_eda=tran_eda[tran_eda.date_flag==1]
print(tran_eda.shape)
tran_flag1=tran_eda
tran_eda.head(12)
#tran_eda= all derived columns
#tran_flag= all flag = 1 with derived columns
#tran_max_date= only max dates

In [ ]:
#get the latest flagged tran date for each observation
tran_max_date=tran_flag1[['I_REC_KEY','DATE']].groupby('I_REC_KEY').tail(1)
tran_max_date.rename(columns={'DATE':'last_tran_date'}, inplace=True)
#tran_date.head(10)
tran_inact=pd.merge(tran_max_date,tran_flag1,left_on=['I_REC_KEY','last_tran_date'],right_on=['I_REC_KEY','DATE'])
print(tran_inact.shape)
tran_inact.head()
tran_inact['inactive_months']=((tran_inact['obs_date2']-tran_inact['last_tran_date'])/np.timedelta64(1, 'M')).astype(int)
tran_inact.inactive_months.value_counts()
#tran_inact.to_csv('tran_inactive_months.txt',header=True,index=False)

In [ ]:
tran_inact.head()

In [ ]:
tran_inact.to_csv('tran_inactive_months.txt',header=True,index=False)

In [ ]:
tran_inact.columns

#### Create new variables in accounts file

In [ ]:
df_sample = pd.read_table('UC_CoBA_NJB/UC_CoBA_ACCT_BAL_MONTHLY_15.txt', nrows=10)
df_sample_size = df_sample.memory_usage(index=True).sum()

my_chunk = (1000000000 / df_sample_size)/10
my_chunk = int(my_chunk//1) # we get the integer part
print (my_chunk)

df_chunk = pd.read_csv('UC_CoBA_ACCT_BAL_MONTHLY_15.txt', chunksize=my_chunk,sep="|")
chunk_list = []  # append each chunk df here
 
# Each chunk is in df format
for chunk in df_chunk: 
    # perform data filtering
    chunk_filter = chunk[chunk['I_REC_KEY'].isin(keys)]
   
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk_filter)
   
# concat the list into dataframe
acct_bal = pd.concat(chunk_list)

acct_bal.drop(['MRKTG_PROD_NAME'],axis=1,inplace=True)

#from datetime import datetime
#acct_bal['DATE']=pd.to_datetime(acct_bal.DATE_KEY)

print(acct_bal.shape)
print(acct_bal.columns)

In [ ]:
import numpy as np
acct_bal['DATE']=pd.to_datetime(acct_bal.DATE_KEY)
acct_bal=acct_bal.sort_values(by=['I_REC_KEY','DATE'])

#ACH_IN
acct_bal['%diff_AVG_MONTHLY_BAL']=acct_bal.groupby(['I_REC_KEY']).AVG_MONTHLY_BAL_AMT.pct_change().replace((np.inf,-np.inf,np.nan), 0)
acct_bal['%diff_LAST_STMT_BAL']=acct_bal.groupby(['I_REC_KEY']).LAST_STMT_BAL_AMT.pct_change().replace((np.inf,-np.inf,np.nan), 0)
acct_bal.drop(['DATE'],1,inplace=True)

acct_bal.to_csv('acct_bal_derv.txt',header=True,index=False)
print(acct_bal.shape)
print(acct_bal.columns)

In [ ]:
acct_bal.shape

In [ ]:
acct_bal=pd.read_csv('acct_bal_derv.txt',header=0,sep=',')
acct_bal.describe()

#### Read customer summary file
#### Merge customer and account summary file

In [ ]:
#Merge cust_sum and acct_bal files and write it as .txt file

#acct_bal=pd.read_csv('acct_bal_derv.txt',header=0,sep=',')
cust_sum=pd.read_csv('cust_summ_train.csv',header=0,sep=',')
cust_acct=pd.merge(cust_sum,acct_bal,on=['I_REC_KEY','DATE_KEY'])
#tran_acct.to_csv('Tran_acct.txt',header=True,index=False)
print(cust_acct.shape)

# Acct + Cust = 2888810 rows ~ 2.8 million rows

In [ ]:
import pandas as pd
cust_sum=pd.read_csv('cust_summ_train.csv',header=0,sep=',',nrows=10)
cust_sum.columns

In [ ]:
cust_acct.columns

#### Merge transaction summary with customer and accounts summary

In [ ]:
#Merge tran_acct and cust_summ_train files and write it as .txt file

#tran_inact=pd.read_csv('last_derv_inact.txt',header=0,sep=',')
tran_inact.drop(['last_tran_date', 'target', 'DATE','obs_date', 'obs_date2','date_flag','DATE_KEY'],1,inplace=True)
tran_cust=pd.merge(tran_inact,cust_acct,on=['I_REC_KEY'])

print(tran_cust.shape)

#Acct + Cust =  2888810 rows ~ 2.8 million rows
#Tran + Acct + Cust = 2502978 ~ 2.5 million rows

In [ ]:
tran_cust.inactive_months.value_counts()

In [ ]:
tran_cust=tran_cust.replace((np.inf,-np.inf,np.nan), 0)
tran_cust.shape

In [ ]:
tran_cust.to_csv('Tran_cust.txt',header=True,index=False)

### Remove outliers - top and bottom 1 percentile

In [ ]:
# Don't change List
donotchange = ['ACCT_KEY', 'I_REC_KEY', 'CUST_KEY','DATE_KEY', 'obs_date',
               'OLDEST_OPEN_DATE','BRANCH_NBR', 'AFFIL_NBR','IXI_CLUSTER_CODE','HH_KEY','target'
               ,'inactive_months']

In [ ]:
# Selecting Numerical columns
tran_cust_n1 = tran_cust.select_dtypes([np.number]) #.columns.to_list()
tran_cust_n2 = tran_cust_n1.columns
num_cols = list(tran_cust_n2)
num_cols

In [ ]:
## Capping and Flooring
num_cols2 = list(set(num_cols).difference(donotchange))
num_cols2

In [ ]:
for col in num_cols2:
        percentiles = tran_cust_out[col].quantile([0.01,0.99]).values
        tran_cust_out[col] = np.clip(tran_cust_out[col], percentiles[0], percentiles[1])
        
# Try1 - 1% and 99%
# Try2 - 5% and 99%

In [ ]:
#Outlier treatment
tran_cust_out=tran_cust
for col in num_cols:
    percentiles = tran_cust_out[col].quantile([0.00,0.99]).values
    tran_cust_out[col] = np.clip(tran_cust_out[col], percentiles[0], percentiles[1])

In [13]:
tran_cust_out=pd.read_csv('capnfloor_acct_cust_L1U99.txt',header=0,sep=',')
tran_cust_out.drop(['Unnamed: 0'],1,inplace=True)
print(tran_cust_out.inactive_months.value_counts())
print(tran_cust_out.target.value_counts())
tran_cust_out.head()

(2502958, 33)


,ACH_IN_MTD_AMT,ACH_IN_MTD_QTY,ACH_OUT_MTD_AMT,ACH_OUT_MTD_QTY,CHK_WRITTEN_MTD_QTY,DEBIT_CARD_MTD_AMT,DEBIT_CARD_MTD_QTY,MOBILE_STD_DEP_QTY,%diff_ACH_IN,diff_ACH_IN_QTY,...,CHECKING_BAL_AMT,CREDIT_CARD_BAL_AMT,OLDEST_OPEN_DATE,LAST_DIRECT_DEPOSIT_AMT,target,AVG_MONTHLY_BAL_AMT,LAST_STMT_BAL_AMT,%diff_AVG_MONTHLY_BAL,%diff_LAST_STMT_BAL,recency
0,1000.00,1.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,...,678.0,896.0,1973-07-01,4116.0,0,59.0,-2.0,-0.797251,-1.250000,553
1,1772.00,4.0,2.0,2.0,2.0,418.99,1.0,0.0,0.000000,0.0,...,3273.0,0.0,1991-11-21,443.0,0,3273.0,3225.0,-0.255967,-0.106648,334
2,7004.02,3.0,0.0,0.0,5.0,0.00,0.0,0.0,-0.448873,-1.0,...,5673.0,0.0,1991-10-20,3388.0,0,4164.0,4678.0,-0.370045,-0.516985,331
3,3897.19,4.0,0.0,0.0,10.0,0.00,0.0,0.0,1.110022,1.0,...,2244.0,3671.0,1991-11-07,647.0,0,2244.0,4191.0,0.909787,2.498331,325
4,7662.69,2.0,2309.6,5.0,1.0,0.00,0.0,0.0,0.450528,1.0,...,6299.0,0.0,1991-08-30,5284.0,0,6299.0,7704.0,0.730970,0.500292,336


In [ ]:
tran_cust_out.describe().to_csv('tran_cust_summary_stats.csv',header=True)
tran_cust_out.describe()

### One Hot encoding for categorical variables

In [ ]:
#Dropping raw variables to check feature importance
tran_cust_d=tran_cust_out.drop(['ACH_IN_MTD_AMT', 'ACH_IN_MTD_QTY','ACH_OUT_MTD_AMT', 'ACH_OUT_MTD_QTY', 'CHECK_CASHING_MTD_AMT',
                         'CHECK_CASHING_MTD_QTY','CHK_WRITTEN_MTD_AMT', 'CHK_WRITTEN_MTD_QTY','DEBIT_CARD_MTD_AMT',
                        'DEBIT_CARD_MTD_QTY', 'AVG_MONTHLY_BAL_AMT', 'LAST_STMT_BAL_AMT'], axis=1)

tran_cust_d.columns

In [15]:
df_dummy = pd.get_dummies(tran_cust_out, drop_first=True)
df_dummy.columns

df_dummy.to_csv('capfloor/recency/tran_cust_dummies.txt',header=True,index=False)

Index(['ACH_IN_MTD_AMT', 'ACH_IN_MTD_QTY', 'ACH_OUT_MTD_AMT',
       'ACH_OUT_MTD_QTY', 'CHK_WRITTEN_MTD_QTY', 'DEBIT_CARD_MTD_AMT',
       'DEBIT_CARD_MTD_QTY', 'MOBILE_STD_DEP_QTY', '%diff_ACH_IN',
       'diff_ACH_IN_QTY', '%diff_ACH_OUT', '%diff_CHECK_WRITTEN',
       'diff_CHECK_WRITTEN', '%diff_DEBIT_CARD', 'diff_DEBIT_CARD_QTY',
       'CHK_WRITTEN_per_trans', 'inactive_months', 'CONS_LOAN_BAL_AMT',
       'CONS_LOAN_WAR_PCT', 'CONS_DEPOSIT_ACCT_QTY', 'CONS_DEPOSIT_BAL_AMT',
       'MORTGAGE_BAL_AMT', 'SAVINGS_BAL_AMT', 'CHECKING_BAL_AMT',
       'CREDIT_CARD_BAL_AMT', 'LAST_DIRECT_DEPOSIT_AMT', 'target',
       'AVG_MONTHLY_BAL_AMT', 'LAST_STMT_BAL_AMT', '%diff_AVG_MONTHLY_BAL',
       '%diff_LAST_STMT_BAL', 'recency', 'DIRECT_DEP_IND_Y',
       'ACTIVE_CHK_IND_Y', 'HABITUAL_OD_IND_Y'],
      dtype='object')

In [ ]:
df_dummy=pd.read_csv('capfloor/tran_cust_dummies.txt',header=0,sep=",")
df_dummy.drop('Unnamed: 0',1,inplace=True)
print(df_dummy.shape)
df_dummy.head()


### Find Information Value of variables

In [16]:
## Information Value
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 20

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

### Find correlation among Variables

In [ ]:
# correlation for continuous variables
corr=tran_cust_n.corr()
corr_tran=corr['target'].sort_values()
print(corr_tran)
corr.to_csv('capfloor/corr_tran_cust.csv', index=True, header=True)

import matplotlib.pyplot as plt
import seaborn as sns
plt.subplots(figsize=(10,10))
chart=sns.heatmap(corr, vmax=0.9, square=True)
chart.set_yticklabels(chart.get_yticklabels(), rotation=0)
plt.savefig('capfloor/tran_cust_corr.jpg', bbox_inches="tight")

In [17]:
X_train_d=df_dummy.drop(['target'],1)
y_train_d=df_dummy.target

In [18]:
final_iv, IV = data_vars(X_train_d,y_train_d)
final_iv=final_iv.sort_values(['IV'],ascending=False)
#final_iv.to_csv('capfloor/Tran_cust_IV.csv',header=True,index=False)
final_iv

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
182,CHECKING_BAL_AMT,7436.0,19507.0,278083,1120,0.004028,276963,0.995972,0.037626,0.111986,-1.090691,1.044922
175,CHECKING_BAL_AMT,-78.0,115.0,278993,15452,0.055385,263541,0.944615,0.519098,0.106559,1.583394,1.044922
181,CHECKING_BAL_AMT,3779.0,7435.0,278035,1147,0.004125,276888,0.995875,0.038533,0.111956,-1.066599,1.044922
180,CHECKING_BAL_AMT,2125.0,3778.0,278042,1393,0.005010,276649,0.994990,0.046797,0.111859,-0.871426,1.044922
179,CHECKING_BAL_AMT,1232.0,2124.0,278006,1828,0.006575,276178,0.993425,0.061410,0.111669,-0.597959,1.044922
...,...,...,...,...,...,...,...,...,...,...,...,...
139,inactive_months,2.0,11.0,101193,2373,0.023450,98820,0.976550,0.079719,0.039956,0.690719,0.029147
138,inactive_months,0.0,1.0,2401765,27394,0.011406,2374371,0.988594,0.920281,0.960044,-0.042300,0.029147
61,MOBILE_STD_DEP_QTY,0.0,1.0,2360210,28993,0.012284,2331217,0.987716,0.973998,0.942595,0.032773,0.027824
62,MOBILE_STD_DEP_QTY,2.0,2.0,65755,439,0.006676,65316,0.993324,0.014748,0.026410,-0.582629,0.027824


In [19]:
final_iv.to_csv('capfloor/recency/Tran_cust_IV.csv',header=True,index=False)

In [ ]:
X_train_d=df_dummy.drop(['target','CHECK_CASHING_MTD_AMT','CHECK_CASHING_MTD_QTY','MOBILE_IMD_DEP_AMT','MOBILE_IMD_DEP_QTY',
'SAFEBOX_BAL_AMT'],1)
y_train_d=df_dummy.target

In [ ]:
#df_dummy=pd.read_csv('tran_cust_dummies.txt',header=0,sep=',')
print(df_dummy.shape)
print(df_dummy.columns)
df_dummy.head()


### Random Forest Feature Importance

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import plot_importance

import datetime
# check run time of the code
start_time=datetime.datetime.now()


model_d=RandomForestClassifier()
%timeit model_d.fit(X_train_d,y_train_d)
print(model_d.get_params())


end_time=datetime.datetime.now()
 
diff = end_time - start_time
 
days, seconds = diff.days, diff.seconds
hours = days * 24 + seconds // 3600
minutes = (seconds % 3600) // 60
seconds = seconds % 60
 
print("Time between dates: %d hours, %d minutes and %d seconds" % (hours, minutes, seconds))

In [ ]:
#feature importance by Random Forest
results_rf_d=pd.DataFrame()
results_rf_d['Variable']=X_train_d.columns
results_rf_d['Importance'] = model_d.feature_importances_
results_rf_d.sort_values(by='Importance',ascending=False,inplace=True)
results_rf_d.to_csv('capfloor/Random Forest_feature_importance_outliers.csv',header=True,index=False)
results_rf_d

### XGBoost Feature Importance

In [ ]:
# fit XGBoost model on training data
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

import datetime
# check run time of the code
start_time=datetime.datetime.now()
print(start_time)

model = XGBClassifier()
%timeit model.fit(X_train_d, y_train_d)
print(model.get_params())


end_time=datetime.datetime.now()
 
diff = end_time - start_time
 
days, seconds = diff.days, diff.seconds
hours = days * 24 + seconds // 3600
minutes = (seconds % 3600) // 60
seconds = seconds % 60
 
print("Time between dates: %d hours, %d minutes and %d seconds" % (hours, minutes, seconds))

In [ ]:
# plot feature importance
XGB=pd.DataFrame()
XGB['Variable']=X_train_d.columns
XGB['Importances'] = model.feature_importances_
XGB.sort_values(by='Importances',ascending=False,inplace=True)
XGB.to_csv('capfloor/XGB_feature_importance_outliers.csv',header=True,index=False)
XGB

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tran_cust_n.drop('target',axis=1), 
           tran_cust_n['target'], test_size=0.30, 
            random_state=101)

In [ ]:
X_train.head()

In [ ]:
X_train.dtypes

### Select significant variables resulting from Feature Importance, IV and correlation analysis

In [2]:
df_dummy=pd.read_csv('capfloor/tran_cust_dummies.txt',header=0,sep=',')
df_dummy_clean_var=df_dummy[['AVG_MONTHLY_BAL_AMT','CHECKING_BAL_AMT','ACTIVE_CHK_IND_Y','ACH_IN_MTD_AMT',
                             'CONS_DEPOSIT_ACCT_QTY','inactive_months','HABITUAL_OD_IND_Y','ACH_IN_MTD_QTY',
                             'CHK_WRITTEN_MTD_QTY','CONS_LOAN_BAL_AMT','CONS_DEPOSIT_BAL_AMT','LAST_STMT_BAL_AMT',
                             '%diff_AVG_MONTHLY_BAL','SAVINGS_BAL_AMT','MOBILE_STD_DEP_QTY','DIRECT_DEP_IND_Y',
                             'DEBIT_CARD_MTD_AMT','ACH_OUT_MTD_QTY','%diff_ACH_IN','diff_ACH_IN_QTY','DEBIT_CARD_MTD_QTY',
                             '%diff_LAST_STMT_BAL','CONS_LOAN_WAR_PCT','diff_DEBIT_CARD_QTY','%diff_DEBIT_CARD',
                             'LAST_DIRECT_DEPOSIT_AMT','diff_CHECK_WRITTEN','%diff_ACH_OUT','ACH_OUT_MTD_AMT',
                             '%diff_CHECK_WRITTEN','MORTGAGE_BAL_AMT','CHK_WRITTEN_per_trans','CREDIT_CARD_BAL_AMT','target'
]]

df_dummy_clean_var.to_csv('capfloor/tran_cust_dummies_signif_var.txt',header=True,index=False)


In [6]:
df_dummy_clean_var=pd.read_csv('capfloor/tran_cust_dummies_signif_var.txt',header=0,sep=',')

X_train=df_dummy_clean_var.drop(['target'],1)
y_train=df_dummy_clean_var.target
X_train.shape

(2502978, 33)

In [7]:
X_train.head()

,AVG_MONTHLY_BAL_AMT,CHECKING_BAL_AMT,ACTIVE_CHK_IND_Y,ACH_IN_MTD_AMT,CONS_DEPOSIT_ACCT_QTY,inactive_months,HABITUAL_OD_IND_Y,ACH_IN_MTD_QTY,CHK_WRITTEN_MTD_QTY,CONS_LOAN_BAL_AMT,...,diff_DEBIT_CARD_QTY,%diff_DEBIT_CARD,LAST_DIRECT_DEPOSIT_AMT,diff_CHECK_WRITTEN,%diff_ACH_OUT,ACH_OUT_MTD_AMT,%diff_CHECK_WRITTEN,MORTGAGE_BAL_AMT,CHK_WRITTEN_per_trans,CREDIT_CARD_BAL_AMT
0,59.0,678.0,1,1000.00,3.0,1,1,1.0,0.0,101589.0,...,0.0,0.000000,4116.0,0.0,0.000000,0.0,0.000000,72717.0,0.0,896.0
1,3273.0,3273.0,1,1772.00,2.0,0,0,4.0,2.0,0.0,...,0.0,5.983167,443.0,2.0,0.000000,2.0,0.000000,0.0,0.0,0.0
2,4164.0,5673.0,1,7004.02,3.0,0,0,3.0,5.0,0.0,...,0.0,0.000000,3388.0,2.0,0.000000,0.0,-0.749441,0.0,0.0,0.0
3,2244.0,2244.0,1,3897.19,2.0,0,0,4.0,10.0,3671.0,...,0.0,0.000000,647.0,7.0,0.000000,0.0,1.121840,0.0,0.0,3671.0
4,6299.0,6299.0,1,7662.69,1.0,0,0,2.0,1.0,0.0,...,0.0,0.000000,5284.0,1.0,-0.487177,2309.6,0.000000,0.0,0.0,0.0


In [5]:
df_dummy_clean_var['target'].value_counts(normalize=True) * 100

0    98.810737
1     1.189263
Name: target, dtype: float64

In [4]:
df_dummy_clean_var['target'].value_counts()

0    2473211
1      29767
Name: target, dtype: int64

### Undersample Data to boost population of targets

In [20]:
from imblearn.under_sampling import RandomUnderSampler
import datetime
from collections import Counter

# check run time of the code
start_time=datetime.datetime.now()
print(start_time)


# define undersample strategy
undersample = RandomUnderSampler(sampling_strategy=0.05, random_state=100)
# fit and apply the transform
X_under, y_under = undersample.fit_resample(X_train_d, y_train_d)
# summarize class distribution
print(Counter(y_under))


end_time=datetime.datetime.now()

diff = end_time - start_time

days, seconds = diff.days, diff.seconds
hours = days * 24 + seconds // 3600
minutes = (seconds % 3600) // 60
seconds = seconds % 60

print("Time between dates: %d hours, %d minutes and %d seconds" % (hours, minutes, seconds))


2020-04-11 15:12:20.325978
Counter({0: 595340, 1: 29767})
Time between dates: 0 hours, 0 minutes and 1 seconds


In [21]:
X_under.to_csv('capfloor/recency/Random_X_train.txt',header=True,index=False)
y_under.to_csv('capfloor/recency/Random_y_train.txt',header=True,index=False)

In [28]:
X_under.shape
y_under.shape

(625107,)

### Split Data into train and test

In [30]:
from sklearn.model_selection import train_test_split
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(
        X_under, y_under,stratify=y_under, test_size=0.872,random_state=100)

X_val_1, X_val_2, y_val_1, y_val_2 = train_test_split(
        X_test_1, y_test_1, stratify=y_test_1, test_size=0.5,random_state=100)
print(Counter(y_train_1))
print(Counter(y_test_1))
print(Counter(y_val_1))
print(Counter(y_val_2))

Counter({0: 76203, 1: 3810})
Counter({0: 519137, 1: 25957})
Counter({0: 259568, 1: 12979})
Counter({0: 259569, 1: 12978})


In [31]:
X_train_1.to_csv('capfloor/recency/X_train.txt',header=True,index=False)
y_train_1.to_csv('capfloor/recency/y_train.txt',header=True,index=False)
X_test_1.to_csv('capfloor/recency/X_total_val.txt',header=True,index=False)
y_test_1.to_csv('capfloor/recency/y_total_val.txt',header=True,index=False)
X_val_1.to_csv('capfloor/recency/X_val_1.txt',header=True,index=False)
y_val_1.to_csv('capfloor/recency/y_val_1.txt',header=True,index=False)
X_val_2.to_csv('capfloor/recency/X_val_2.txt',header=True,index=False)
y_val_2.to_csv('capfloor/recency/y_val_2.txt',header=True,index=False)

### Train set will be used for modelling